<a href="https://colab.research.google.com/github/carlcodecode/Data-Science-II/blob/main/SimpleCNNforTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.cuda.get_device_name(torch.cuda.current_device()))


Tesla T4


In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
train_data = datasets.CIFAR10(root="data", train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1024, shuffle=True)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    # 3 input channels, 6 output channels, 5x5 kernel
    self.conv1 = nn.Conv2d(3, 6, 5)

    # 2x2 max pooling
    self.pool = nn.MaxPool2d(2, 2)

    # 6 input (from the last output), 16 output, 5x5 kernel
    self.conv2 = nn.Conv2d(6, 16, 5)

    #connected layers
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # first convolutional layer
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)

    # second convolutional layer
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    # flatten
    x = torch.flatten(x, 1)   # flatten all dim except batch

    # go through fully connected layers
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)

    return x





In [10]:
import torch.optim as optim

model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

epochs = 10

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for X_batch, y_batch in train_loader:
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)

    # forward pass
    logits = model(X_batch)

    # compute loss
    loss = loss_fn(logits, y_batch)

    # three step mantra
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch {epoch+1}: Loss = {(running_loss / len(train_loader)):.4f}")




Epoch 1: Loss = 2.0590
Epoch 2: Loss = 1.7530
Epoch 3: Loss = 1.6041
Epoch 4: Loss = 1.5094
Epoch 5: Loss = 1.4566
Epoch 6: Loss = 1.4154
Epoch 7: Loss = 1.3833
Epoch 8: Loss = 1.3530
Epoch 9: Loss = 1.3288
Epoch 10: Loss = 1.3022


In [11]:
# testing

model.eval()
correct = 0
total = 0
with torch.no_grad():
  for X_batch, y_batch in test_loader:
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)

    outputs = model(X_batch)

    _, predictions = torch.max(outputs.data, dim=1)
    correct += (predictions == y_batch).sum().item()
    total += y_batch.size(dim=0)

  print(f"Accuracy = {100 * correct / total}")


Accuracy = 51.96
